Importing libs

In [41]:
import pandas as pd
from io import StringIO
import io
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
from sklearn.preprocessing import LabelEncoder

Importing ds

In [42]:
from google.colab import files
uploaded = files.upload()

Saving BarisalFlood.csv to BarisalFlood (3).csv


In [43]:
df = pd.read_csv(io.BytesIO(uploaded['BarisalFlood.csv']))

Processing data

In [44]:
df.drop(columns='Station_Names',inplace=True)
df.drop(columns='Station_Number',inplace=True)
df.drop(columns='X_COR',inplace=True)
df.drop(columns='Y_COR',inplace=True)
df.drop(columns='LATITUDE',inplace=True)
df.drop(columns='LONGITUDE',inplace=True)
df.drop(columns='ALT',inplace=True)
df['Flood?'] = df['Flood?'].fillna(0)
encoder=LabelEncoder()
df['Sl']=encoder.fit_transform(df['Sl'])

Visualizing the data

In [45]:
print(df)

      Sl  Year  Month  Max_Temp  Min_Temp  Rainfall  Relative_Humidity  \
0      0  1949      1      29.4      12.3         0                 68   
1      1  1949      2      33.9      15.2         9                 63   
2      2  1949      3      36.7      20.2         8                 59   
3      3  1949      4      33.9      23.9       140                 71   
4      4  1949      5      35.6      25.0       217                 76   
..   ...   ...    ...       ...       ...       ...                ...   
775  775  2013      8      34.0      26.2       398                 88   
776  776  2013      9      35.0      26.0       312                 88   
777  777  2013     10      34.2      24.4       181                 88   
778  778  2013     11      32.2      17.7         0                 81   
779  779  2013     12      29.5      14.2         0                 82   

     Wind_Speed  Cloud_Coverage  Bright_Sunshine   Period  Flood?  
0      0.453704             0.6         7.8

LSTM Net

In [55]:
X_columns=[x for x in df.columns if x!='Flood?']
features=len(X_columns)
model = Sequential()
model.add(LSTM(100, activation='tanh', return_sequences=True, input_shape=(1, features)))
model.add(LSTM(49, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
history=model.compile(optimizer="rmsprop", loss='binary_crossentropy', metrics=['accuracy'])


In [56]:
grouped=df.groupby('Sl')

In [57]:
for group in grouped:
    df_batch=pd.DataFrame(columns=['Sl','Year','Month','Max_Temp','Min_Temp','Rainfall','Relative_Humidity','Wind_Speed','Cloud_Coverage','Bright_Sunshine','Period','Flood?'])
    for Sl in group:
        filter=df['Sl']==Sl
        for key,item in df[filter].dropna().iterrows():
            df_batch=df_batch.append({'Sl':item['Sl'],'Year':item['Year'],'Month':item['Month'],'Max_Temp':item['Max_Temp'],'Min_Temp':item['Min_Temp'],'Rainfall':item['Rainfall'],'Relative_Humidity':item['Relative_Humidity'],'Wind_Speed':item['Wind_Speed'],'Cloud_Coverage':item['Cloud_Coverage'],'Bright_Sunshine':item['Bright_Sunshine'],'Period':item['Period'],'Flood?':item['Flood?']},ignore_index=True)
    X=df_batch[X_columns]
    X = np.asarray(X).astype(np.float32)
    X = np.resize(X,(X.shape[0],1,X.shape[1]))
    y=df_batch['Flood?']
    y = np.asarray(y).astype(np.float32)
    print("\n",X)
    model.fit(X,y,batch_size=len(X),epochs=10)


 [[[0.000000e+00 1.949000e+03 1.000000e+00 2.940000e+01 1.230000e+01
   0.000000e+00 6.800000e+01 4.537037e-01 6.000000e-01 7.831915e+00
   1.949010e+03]]]
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  after removing the cwd from sys.path.


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 16ms/step - loss: 2.2152e-06 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 14ms/step - loss: 2.1694e-06 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 14ms/step - loss: 2.1246e-06 - accuracy: 1.0000

 [[[5.70000e+02 1.99600e+03 7.00000e+00 3.50000e+01 2.57000e+01
   3.07000e+02 8.80000e+01 1.60000e+00 5.90000e+00 3.30000e+00
   1.99607e+03]]]
Epoch 1/10
1/1 [==============================] - 0s 30ms/step - loss: 8.1366 - accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 21ms/step - loss: 2.0394e-06 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 21ms/step - loss: 2.0390e-06 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 14ms/step - loss: 2.0387e-06 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 17ms/step - loss: 2.0383e-06 - accuracy: 1.0000
Epoch

In [58]:
import tensorflow as tf

In [59]:
new_bin_cross_entropy = tf.keras.losses.BinaryCrossentropy()
result = new_bin_cross_entropy(X,y).numpy()
print(result)

6942.6577
